In [93]:
#Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import time
import json
from pprint import pprint
from scipy.stats import ttest_ind
from datetime import datetime
from config import yelp_key

In [2]:
county_datafile = "data/LOS_ANGELES_COUNTY_RESTAURANT_AND_MARKET_INSPECTIONS.csv"

county_data = pd.read_csv(county_datafile)
county_data.count()

ACTIVITY DATE           199950
OWNER ID                199950
OWNER NAME              199950
FACILITY ID             199950
FACILITY NAME           199950
RECORD ID               199950
PROGRAM NAME            199950
PROGRAM STATUS          199950
PROGRAM ELEMENT (PE)    199950
PE DESCRIPTION          199950
FACILITY ADDRESS        199950
FACILITY CITY           199950
FACILITY STATE          199950
FACILITY ZIP            199950
SERVICE CODE            199950
SERVICE DESCRIPTION     199950
SCORE                   199950
GRADE                   199895
SERIAL NUMBER           199950
EMPLOYEE ID             199950
dtype: int64

In [162]:
columns = ['ACTIVITY DATE','FACILITY ID','FACILITY NAME','PROGRAM STATUS','PE DESCRIPTION','FACILITY ADDRESS','FACILITY CITY','FACILITY ZIP','SCORE','GRADE']
cleaned_county_data = county_data[columns]
cleaned_county_data['ACTIVITY DATE'] = pd.to_datetime(cleaned_county_data['ACTIVITY DATE'])
cleaned_county_data.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,ACTIVITY DATE,FACILITY ID,FACILITY NAME,PROGRAM STATUS,PE DESCRIPTION,FACILITY ADDRESS,FACILITY CITY,FACILITY ZIP,SCORE,GRADE
0,2016-06-22,FA0179753,TEA BAR,ACTIVE,RESTAURANT (31-60) SEATS HIGH RISK,954 E ALOSTA AVE,AZUSA,91702,93,A
1,2016-09-07,FA0239319,SHEFA,ACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,7275 MELROSE AVE,LOS ANGELES,90046,90,A
2,2016-09-09,FA0071154,SUBWAY,ACTIVE,RESTAURANT (0-30) SEATS MODERATE RISK,28788 THE OLD RD,VALENCIA,91355,98,A
3,2016-04-01,FA0144494,AFC SUSHI @ VONS #3138,ACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,16550 SOLEDAD CANYON RD,CANYON COUNTRY,91387-3215,99,A
4,2016-04-01,FA0005693,ALBERTS MEXICAN FOOD,INACTIVE,RESTAURANT (0-30) SEATS HIGH RISK,2355 PECK RD,SOUTH EL MONTE,91733,92,A


In [164]:
cleaned_county_data.sort_values(by = ["ACTIVITY DATE","FACILITY ID","FACILITY ADDRESS"],inplace = True,ascending=True)
cleaned_county_data.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,ACTIVITY DATE,FACILITY ID,FACILITY NAME,PROGRAM STATUS,PE DESCRIPTION,FACILITY ADDRESS,FACILITY CITY,FACILITY ZIP,SCORE,GRADE
94,2016-04-01,FA0002213,RANCHITO MILLAN,ACTIVE,RESTAURANT (61-150) SEATS HIGH RISK,5803 S SOTO ST,HUNTINGTON PARK,90255,94,A
12,2016-04-01,FA0004764,BALDWIN OIL,ACTIVE,"FOOD MKT RETAIL (1-1,999 SF) LOW RISK",13758 LOS ANGELES ST,BALDWIN PARK,91706,98,A
14,2016-04-01,FA0004876,BANQUET KITCHEN,ACTIVE,RESTAURANT (151 + ) SEATS HIGH RISK,135 N GRAND AVE,LOS ANGELES,90012,97,A
5,2016-04-01,FA0005575,ALL STAR PARK COMPANY,INACTIVE,RESTAURANT (0-30) SEATS LOW RISK,17911 S BELLFLOWER BLVD,BELLFLOWER,90706,91,A
17,2016-04-01,FA0005633,BIG O LIQUIOR,INACTIVE,"FOOD MKT RETAIL (1-1,999 SF) LOW RISK",4626 S VERMONT AVE,LOS ANGELES,90037,95,A


In [165]:
unique_county_data = cleaned_county_data.drop_duplicates(subset = ['FACILITY ID'], keep = 'last')
unique_county_data.count()

ACTIVITY DATE       44581
FACILITY ID         44581
FACILITY NAME       44581
PROGRAM STATUS      44581
PE DESCRIPTION      44581
FACILITY ADDRESS    44581
FACILITY CITY       44581
FACILITY ZIP        44581
SCORE               44581
GRADE               44563
dtype: int64

In [166]:
unique_county_data = unique_county_data.loc[(unique_county_data['PROGRAM STATUS'] != "INACTIVE"),:]
unique_county_data.count()

ACTIVITY DATE       36474
FACILITY ID         36474
FACILITY NAME       36474
PROGRAM STATUS      36474
PE DESCRIPTION      36474
FACILITY ADDRESS    36474
FACILITY CITY       36474
FACILITY ZIP        36474
SCORE               36474
GRADE               36462
dtype: int64

In [167]:
unique_county_data = unique_county_data[unique_county_data['PE DESCRIPTION'].str.contains("RESTAURANT")]
unique_county_data.count()

ACTIVITY DATE       26525
FACILITY ID         26525
FACILITY NAME       26525
PROGRAM STATUS      26525
PE DESCRIPTION      26525
FACILITY ADDRESS    26525
FACILITY CITY       26525
FACILITY ZIP        26525
SCORE               26525
GRADE               26515
dtype: int64

In [168]:
unique_county_data = unique_county_data.dropna(how='any')
unique_county_data.count()

ACTIVITY DATE       26515
FACILITY ID         26515
FACILITY NAME       26515
PROGRAM STATUS      26515
PE DESCRIPTION      26515
FACILITY ADDRESS    26515
FACILITY CITY       26515
FACILITY ZIP        26515
SCORE               26515
GRADE               26515
dtype: int64

In [169]:
nofastfood_df = unique_county_data[~unique_county_data['FACILITY NAME'].str.contains("#")]
nofastfood_df.count()

ACTIVITY DATE       22512
FACILITY ID         22512
FACILITY NAME       22512
PROGRAM STATUS      22512
PE DESCRIPTION      22512
FACILITY ADDRESS    22512
FACILITY CITY       22512
FACILITY ZIP        22512
SCORE               22512
GRADE               22512
dtype: int64

In [156]:
sample_data = nofastfood_df.sample(n=1000)
sample_data

,ACTIVITY DATE,FACILITY ID,FACILITY NAME,PROGRAM STATUS,FACILITY ADDRESS,FACILITY CITY,FACILITY ZIP,SCORE,GRADE
128196,2017-11-20,FA0053244,UNION GRIND,ACTIVE,1000 E VICTORIA ST,CARSON,90747,96,A
166589,2018-05-14,FA0003747,ANDERS JUICE BAR,ACTIVE,7230 FIRESTONE BLVD,DOWNEY,90241,90,A
166856,2018-05-15,FA0003052,AJISEN RAMEN,ACTIVE,18253 COLIMA RD STE 104,ROWLAND HEIGHTS,91748,94,A
193503,2018-09-06,FA0002309,7-ELEVEN,ACTIVE,504 S CITRUS AVE,AZUSA,91702,97,A
159096,2018-04-13,FA0053294,MINI MERCADITO IN E.L.A.,ACTIVE,3425 E 1ST ST STE 132,LOS ANGELES,90063,96,A
144814,2018-02-12,FA0073844,WENDY'S MARKET,ACTIVE,3354 EAGLE ROCK BLVD,LOS ANGELES,90065,83,B
155020,2018-03-28,FA0034085,STANLEY'S LIQUOR,ACTIVE,1803 S PACIFIC AVE,SAN PEDRO,90731,91,A
159480,2018-04-16,FA0038865,THE LA HOTEL DOWNTOWN - ZIRAN,ACTIVE,333 S FIGUEROA ST # 103,LOS ANGELES,90071,90,A
195562,2018-09-13,FA0242434,SEVEN SEAS GOURMET FOODS,ACTIVE,318 N CENTRAL AVE,GLENDALE,91203,98,A
151868,2018-03-14,FA0023259,MADRONA CAR WASH,ACTIVE,3405 SEPULVEDA BLVD,TORRANCE,90503,97,A


In [171]:
#headers = {"Authorization": "Bearer "+yelp_key}
#params = {"term": unique_county_data['FACILITY NAME'].iloc[0],"location": unique_county_data['FACILITY ZIP'].iloc[0]}
#target_url = "https://api.yelp.com/v3/businesses/search"

#yelp_reviews = requests.get(target_url, params, headers=headers).json()
#yelp_reviews

In [157]:
headers = {"Authorization": "Bearer "+ yelp_key}
target_url = "https://api.yelp.com/v3/businesses/search"

final_df = pd.DataFrame()
name = []
rating = []
review_count = []
price = []

#for index, row in sample_data.iloc[:10,:].iterrows():
for index, row in sample_data.iterrows():
    restr_name = row['FACILITY NAME']
    restr_zip = row['FACILITY ZIP']
    restr_addr = row['FACILITY ADDRESS']
    restr_city = row['FACILITY CITY']
    params = {"term": restr_name,"location": str(restr_loc + ", " + restr_city + ", "+restr_zip)}
    
    yelp_response = requests.get(target_url, params, headers=headers)
    
    if yelp_response.status_code != requests.codes.ok:
        print(yelp_response.text)
    else:
        yelp_data = yelp_response.json()
        try:
            name.append(yelp_data['businesses'][0]['name'])
            #print(yelp_data['businesses'][0]['name'])
        except (KeyError, IndexError):
            name.append("NA")
            #print("Missing Name...")
        try:
            rating.append(yelp_data['businesses'][0]['rating'])
            #print(yelp_data['businesses'][0]['rating'])
        except (KeyError, IndexError):
            rating.append("NA")
            #print("Missing Rating...")
        try:
            review_count.append(yelp_data['businesses'][0]['review_count'])
            #print(yelp_data['businesses'][0]['review_count'])
        except (KeyError, IndexError):
            review_count.append("NA")
            #print("Missing Reviews...")    
        try:    
            price.append(yelp_data['businesses'][0]['price'])
            #print(yelp_data['businesses'][0]['price'])
        except (KeyError, IndexError):
            price.append("NA")
            #print("Missing Price...")


In [159]:
final_df['Name'] = name
final_df['Rating'] = rating
final_df['Reviews'] = review_count
final_df['Price'] = price
final_df.count()

Name       1000
Rating     1000
Reviews    1000
Price      1000
dtype: int64

In [160]:
final_df['Reviews'] = pd.to_numeric(final_df['Reviews'],errors = 'coerce').fillna(0).astype(np.int64)
q1 = np.percentile(final_df['Reviews'],25)
q1

19.0

In [161]:
percentiled_df = final_df.loc[final_df['Reviews'] > q1, :]
percentiled_df

,Name,Rating,Reviews,Price
1,Mattern Sausage & Deli,4.5,534,$
2,Ajisen Ramen,3.5,404,$$
5,Belcampo Burger DTLA,3.5,502,$$
7,The L.A. Grand Hotel Downtown,3,274,$$$
9,Madrona Car Wash,3,297,NA
11,See's Candies,4,86,$$
12,Cinnabon,3,29,$
13,Tlaco Taco,3.5,27,$
15,Hop Li Seafood Restaurant,3.5,323,$$
16,Palms Thai Restaurant,4,2504,$$


In [170]:
# Save Data to csv
percentiled_df.to_csv("final_output_data.csv")